In [7]:
import boto3 
from botocore.exceptions import ClientError
import os 
import tempfile
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

def download_faiss_from_s3(folder):
    bucket_name = os.getenv('AWS_S3_UPLOAD_BUCKET')
    session = boto3.Session(profile_name=os.getenv('AWS_PROFILE'))
    s3 = session.client('s3')
    tmp_dir = tempfile.mkdtemp()

    for filename in ["index.faiss", "index.pkl"]:
        s3_key = f"{folder}/{filename}"
        local_path = os.path.join(tmp_dir, filename)
        print("S3 download:", bucket_name, s3_key, "->", local_path)
        s3.download_file(bucket_name, s3_key, local_path)

    return os.path.join(folder, tmp_dir)


session_id = "17decbd4-92e7-4d9d-9736-92841f367d8c"
faiss_folder = download_faiss_from_s3(f"faiss_index/{session_id}")
print(faiss_folder)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.load_local(faiss_folder, embeddings, allow_dangerous_deserialization=True)

print(vector_store)


S3 download: s3-thm-demo faiss_index/17decbd4-92e7-4d9d-9736-92841f367d8c/index.faiss -> C:\Users\indra\AppData\Local\Temp\tmplgj3cg1a\index.faiss
S3 download: s3-thm-demo faiss_index/17decbd4-92e7-4d9d-9736-92841f367d8c/index.pkl -> C:\Users\indra\AppData\Local\Temp\tmplgj3cg1a\index.pkl
C:\Users\indra\AppData\Local\Temp\tmplgj3cg1a


d:\Code\Python\aws-genai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import logging
import watchtower 
import boto3 
import os 

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

session = boto3.Session(profile_name=os.getenv('AWS_PROFILE'), region_name="ap-south-1")
client = session.client('logs')

cloudwatch_handler = watchtower.CloudWatchLogHandler(
    boto3_client=client,
    log_group="Py-test",
    stream_name="test"
)
logger.addHandler(cloudwatch_handler)

logger.info("Test log message")


In [ ]:
def put_metric(name, value, unit="Count"):
    # cloudwatch = boto3.client("cloudwatch")

    session = boto3.Session(profile_name=os.getenv('AWS_PROFILE'), region_name="ap-south-1")
    cloudwatch = session.client('cloudwatch')

    cloudwatch.put_metric_data(
        Namespace="GenAI",
        MetricData=[{
            "MetricName": name,
            "Value": value,
            "Unit": unit
        }]
    )

put_metric('Data entries', 2)

NoRegionError: You must specify a region.